In [ ]:
import cv2
import mediapipe as mp

# initiallizating hand recogintion

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode = False,
    max_num_hands = 1,
    min_detection_confidence = 0.7,
    min_tracking_confidence = 0.7
)

mp_draw = mp.solutions.drawing_utils

def detection(hand_landmarks, hand_label):
    fingers = []
    landmarks = hand_landmarks.landmark
    tip_ids = [4, 8, 12, 16, 20]

    # Thumb detection: logic depends on whether it's Left or Right hand
    if hand_label == "Right":
        if landmarks[tip_ids[0]].x < landmarks[tip_ids[0] - 1].x:
            fingers.append(1)
        else:
            fingers.append(0)
    else:  # Left hand
        if landmarks[tip_ids[0]].x > landmarks[tip_ids[0] - 1].x:
            fingers.append(1)
        else:
            fingers.append(0)

    # Other four fingers
    for id in range(1, 5):
        if landmarks[tip_ids[id]].y < landmarks[tip_ids[id] - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)

    return fingers






# webcam feed.

capture = cv2.VideoCapture(0)

while True:
    ret, frame = capture.read()
    if not ret :
        break

    frame = cv2.flip(frame ,1) # flipping to self veiw

    rgb_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) # conversion to rgb type

    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks :
       for idx, hand_landmarks in enumerate(results.multi_hand_landmarks) :
          
          mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

          hand_label = results.multi_handedness[idx].classification[0].label  # 'Left' or 'Right'
          confidence = results.multi_handedness[idx].classification[0].score  # Confidence (0.0 - 1.0)

          h , w , _ = frame.shape
          landmarks = hand_landmarks.landmark


          x_list = [int(lm.x * w) for lm in landmarks]
          y_list = [int(lm.y * h) for lm in landmarks]

          x_min , x_max = min(x_list) , max(x_list)
          y_min, y_max = min(y_list) , max(y_list)

          
          cv2.rectangle(frame, (x_min - 20, y_min - 20), (x_max + 20, y_max + 20), (255, 0, 0), 2)


          #cv2.putText(frame, display_text, (x_min - 20, y_max + 40), 
           #      cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)


          finger_states = detection(hand_landmarks,hand_label)
          total_fingers = sum(finger_states)

          if total_fingers == 0:
             gesture = "Fist" 

          elif total_fingers == 5:
             gesture = "Open Palm"
          elif finger_states == [0, 1, 1, 0, 0]:
                gesture = "Peace Sign"   
          elif finger_states == [1,0,0,0,0]:
                gesture = "Thumbs Up"    
          elif finger_states == [1,1,0,0,1]:
                gesture = "Rock Sign"       
          else :
             gesture = f"{total_fingers} fingers up"


          display_text = f"Fingers : {total_fingers} gesture : {gesture}"

          cv2.putText(frame, f"{hand_label} Hand", (x_min - 20, y_min - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

          cv2.putText(frame, display_text, (x_min - 20, y_max + 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)


    # video frame
    cv2.imshow("Hand Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    
    



capture.release()
cv2.destroyAllWindows


c:\Users\Admin\miniconda3\envs\palm_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<function destroyAllWindows>

: 